<a href="https://colab.research.google.com/github/BOHDANMARCEN/barkUA/blob/main/Bark_TTS_Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Інсталяція необхідних бібліотек
!pip install git+https://github.com/suno-ai/bark.git
!pip install ipywidgets encodec torchaudio


  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-10k_cib2
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-10k_cib2
  Resolved https://github.com/suno-ai/bark.git to commit f4f32d4cd480dfec1c245d258174bc9bde3c2148
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 38.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 5.8 MB/s eta 0:00:00
  Created wheel for suno-bark: filename=suno_bark-0.0.1a0-py3-none-any.whl size=2567357 sha256=54c4d22dbd9fd54ecaa1a8c08a34af669387fd4216f120bbafc3ec7928c50bf2
  Stored in directory: /tmp/pip-ephem-wheel-cache-ubc

In [3]:
# Імпортуємо необхідні модулі
import numpy as np
from bark import SAMPLE_RATE, generate_audio, preload_models
from bark.generation import load_codec_model, codec_decode
from encodec import EncodecModel
import torch
import torchaudio
from IPython.display import Audio, display
import os
from scipy.io import wavfile
import tempfile

# Маппінг української до латиниці
ua_to_latin = {
    'а': 'a', 'б': 'b', 'в': 'v', 'г': 'h', 'ґ': 'g', 'д': 'd', 'е': 'e',
    'є': 'ye', 'ж': 'zh', 'з': 'z', 'и': 'y', 'і': 'i', 'ї': 'yi', 'й': 'j',
    'к': 'k', 'л': 'l', 'м': 'm', 'н': 'n', 'о': 'o', 'п': 'p', 'р': 'r',
    'с': 's', 'т': 't', 'у': 'u', 'ф': 'f', 'х': 'kh', 'ц': 'ts', 'ч': 'ch',
    'ш': 'sh', 'щ': 'shch', 'ь': '', 'ю': 'yu', 'я': 'ya',
    'А': 'A', 'Б': 'B', 'В': 'V', 'Г': 'H', 'Ґ': 'G', 'Д': 'D', 'Е': 'E',
    'Є': 'Ye', 'Ж': 'Zh', 'З': 'Z', 'И': 'Y', 'І': 'I', 'Ї': 'Yi', 'Й': 'J',
    'К': 'K', 'Л': 'L', 'М': 'M', 'Н': 'N', 'О': 'O', 'П': 'P', 'Р': 'R',
    'С': 'S', 'Т': 'T', 'У': 'U', 'Ф': 'F', 'Х': 'Kh', 'Ц': 'Ts', 'Ч': 'Ch',
    'Ш': 'Sh', 'Щ': 'Shch', 'Ь': '', 'Ю': 'Yu', 'Я': 'Ya'
}

def transliterate_ukrainian(text):
    result = ''
    for char in text:
        result += ua_to_latin.get(char, char)
    return result

class VoiceCloner:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = None

    def load_model(self):
        if self.model is None:
            self.model = EncodecModel.encodec_model_24khz()
            self.model.set_target_bandwidth(6.0)
            self.model.eval()
            self.model.to(self.device)

    def extract_semantic_features(self, audio_path):
        wav, sr = torchaudio.load(audio_path)
        if sr != 24000:
            wav = torchaudio.transforms.Resample(sr, 24000)(wav)
        if wav.shape[0] == 2:
            wav = wav.mean(0, keepdim=True)
        wav = wav.to(self.device)
        with torch.no_grad():
            encoded_frames = self.model.encode(wav.unsqueeze(0))
            codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)
        return codes.cpu().numpy()

class BarkTTSInterface:
    def __init__(self):
        self.audio_array = None
        self.cloned_voice_features = None
        self.voice_cloner = VoiceCloner()
        preload_models()

    def generate_audio(self, text, voice_type='Попередньо записані голоси', voice_label=None, audio_file=None):
        try:
            transliterated_text = transliterate_ukrainian(text)
            if voice_type == 'Попередньо записані голоси':
                self.audio_array = generate_audio(transliterated_text, history_prompt=voice_label)
            else:
                if self.cloned_voice_features is None:
                    raise ValueError("Голос не клонувався.")
                self.audio_array = generate_audio(transliterated_text, history_prompt=self.cloned_voice_features)
            return Audio(self.audio_array, rate=SAMPLE_RATE)
        except Exception as e:
            print(f"Помилка генерації: {e}")
            return None

    def clone_voice(self, audio_path):
        try:
            self.voice_cloner.load_model()
            self.cloned_voice_features = self.voice_cloner.extract_semantic_features(audio_path)
            print("Голос успішно клоновано.")
        except Exception as e:
            print(f"Помилка клонування: {e}")

# Використання інтерфейсу
tts = BarkTTSInterface()

# 1. Клонування голосу
# Завантажте аудіо-файл у Colab, наприклад, через панель "Файли", і вкажіть його шлях нижче
audio_file = '/content/your_audio_file.wav'  # Замініть на ваш шлях
tts.clone_voice(audio_file)

# 2. Генерація аудіо
text = "Привіт, це тестове повідомлення."
voice_label = 'v2/en_speaker_0'  # Замініть на бажаний голос
audio_widget = tts.generate_audio(text, voice_type='Попередньо записані голоси', voice_label=voice_label)

# Відтворення аудіо
if audio_widget:
    display(audio_widget)


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Помилка клонування: Failed to open the input "/content/your_audio_file.wav" (No such file or directory).
Exception raised from get_input_format_context at /__w/audio/audio/pytorch/audio/src/libtorio/ffmpeg/stream_reader/stream_reader.cpp:42 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x96 (0x7978adf81446 in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::string const&) + 0x64 (0x7978adf2b6e4 in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x42134 (0x7978adebd134 in /usr/local/lib/python3.10/dist-packages/torio/lib/libtorio_ffmpeg4.so)
frame #3: torio::io::StreamingMediaDecoder::StreamingMediaDecoder(std::string const&, std::optional<std::string> const&, std::optional<std::map<std::string, std::string, std::less<std::string>, std::allocator<std::pair<std::string const, std::string> > > > const&) + 0x1

100%|██████████| 8/8 [02:37<00:00, 19.65s/it]
